In [2]:
!git clone https://github.com/NVIDIA-AI-IOT/jetcam

Cloning into 'jetcam'...
remote: Enumerating objects: 191, done.
remote: Total 191 (delta 0), reused 0 (delta 0), pack-reused 191
Receiving objects: 100% (191/191), 41.34 KiB | 1.38 MiB/s, done.
Resolving deltas: 100% (78/78), done.


In [1]:

!ls -ltrh /dev/video*

crw-rw---- 1 root video 81, 0 Apr 15 05:22 /dev/video0


In [2]:
!ls -ltrh /dev/video*

# USB Camera (Logitech C270 webcam)
from jetcam.usb_camera import USBCamera
camera = USBCamera(width=224, height=224, capture_device=0) # confirm the capture_device number

# CSI Camera (Raspberry Pi Camera Module V2)
# from jetcam.csi_camera import CSICamera
# camera = CSICamera(width=224, height=224)

camera.running = True
print("camera created")

crw-rw---- 1 root video 81, 0 Apr 15 05:22 /dev/video0
camera created


In [3]:
import os

blocked_dir = 'dataset/saved'
# free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
#     os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created because they already exist')

Directories not created because they already exist


In [4]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
button_layout = widgets.Layout(width='128px', height='64px')
# free_button = widgets.Button(description='save', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='save', button_style='danger', layout=button_layout)
# free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

# display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

In [5]:
import numpy as  np
# import numpy as np
from PIL import Image
def rotate(front,back,top,bottom,left,right,itr,i,c):
    
    sz=front.shape[0]
    
    if itr == 1:
        front,back,top,bottom,left,right = right,left,np.rot90(top,3),np.rot90(bottom,1),front,back
    elif itr == 2 :
        front,back,top,bottom,left,right = top,np.rot90(bottom,2),np.rot90(back,2),front,np.rot90(left,3),np.rot90(right,1)


    if c == 0:
        temp = np.copy(front[i])
        front[i]= right[i]
        right[i]=back[i]
        back[i]=left[i]
        left[i]=temp
        if i == 0:
            top = np.rot90(top,3)
        elif i == sz-1:
            bottom = np.rot90(bottom,1)
    elif c == 1:
        temp = np.copy(front[i])
        front[i]= left[i]
        left[i]=back[i]
        back[i]=right[i]
        right[i]=temp
        if i == 0:
            top = np.rot90(top,1)
        elif i == sz-1:
            bottom = np.rot90(bottom,3)
    elif c == 2:
        temp = np.copy(front[:,i])
        front[:,i]= bottom[:,i]
        bottom[:,i]=np.flip(back[:,sz-i-1])
        back[:,sz-i-1]=np.flip(top[:,i])
        top[:,i]=temp
        if i == 0:
            left = np.rot90(left,1)
        elif i == sz-1:
            right = np.rot90(right,3)
    else:
        temp = np.copy(front[:,i])
        front[:,i]= top[:,i]
        top[:,i]=np.flip(back[:,sz-i-1])
        back[:,sz-i-1]=np.flip(bottom[:,i])
        bottom[:,i]=temp
        if i == 0:
            left = np.rot90(left,3)
        elif i == sz-1:
            right = np.rot90(right,1)

    if itr == 1:
        temp  = np.copy(front)
        front = left
        left = back
        back = right
        top = np.rot90(top,1)
        bottom = np.rot90(bottom,3)
        right = temp
    elif itr == 2:
        temp = np.copy(front)
        front = bottom
        bottom = np.rot90(back,2)
        back = np.rot90(top,2)
        top = temp
        left = np.rot90(left,1)
        right = np.rot90(right,3)

    return front,back,top,bottom,left,right

def logistic_key(x, r, size):
    key = []
    for i in range(size):   
        x = r*x*(1-x)   # The logistic equation
        key.append(int((x*pow(10, 16))%256))    # Converting the generated number between 0 to 255
    return key





In [6]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

# def save_free():
#     global free_dir, free_count
#     save_snapshot(free_dir)
#     free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    encrypt()
    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
# free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

In [7]:
import os
def encrypt(path = r"dataset/saved/",savepath = r"dataset/encrypted/"):
    for filename in os.listdir(path):
        ext = os.path.splitext(filename)[-1].lower()
        allowed = ['.jpg','.png','jpeg']
        if ext not in allowed:
            continue
        img = Image.open(f'{path}{filename}')
        img = np.array(img)

        finalencimage  = np.copy(img)
        shrya = np.copy(img)
        for ii in range(3): 
            img  = np.copy(shrya[:,:,ii])

            front = np.zeros(img.shape)
            back = np.zeros(img.shape)
            top = np.zeros(img.shape)
            bottom = np.zeros(img.shape)
            left = np.zeros(img.shape)
            right = np.zeros(img.shape)

            for i in range(img.shape[0]):
                for j in range(img.shape[1]):
                    pix = [int(k) for k in list('{0:08b}'.format(img[i,j]))]
                    front[i][j] = pix[0]
                    back[i][j]=pix[1]
                    top[i][j]=pix[2]
                    bottom[i][j]=pix[3]
                    left[i][j]=pix[4]
                    right[i][j]=pix[5]

            imgn = np.copy(img)

            key = logistic_key(0.01, 3.85, img.shape[0]*img.shape[1])

            k=0

            for i in range(3):
                for j in range(front.shape[0]):
                    front,back,top,bottom,left,right = rotate(front,back,top,bottom,left,right,i,j,key[k]%4)
                    k+=1

            ke=0
            for i in range(img.shape[0]):
                for j in range(img.shape[1]):
                    imgn[i][j] = 0
                    pix = [int(k) for k in list('{0:08b}'.format(img[i,j]))]
                    imgn[i][j] += (np.uint64(front[i][j])<<np.uint64(7))
                    imgn[i][j] += (np.uint64(back[i][j])<<np.uint64(6))
                    imgn[i][j] += (np.uint64(top[i][j])<<np.uint64(5))
                    imgn[i][j] += (np.uint64(bottom[i][j])<<np.uint64(4))
                    imgn[i][j] += (np.uint64(left[i][j])<<np.uint64(3))
                    imgn[i][j] += (np.uint64(right[i][j])<<np.uint64(2))
                    imgn[i][j] += (pix[6]<<1)
                    imgn[i][j] += (pix[7])
                    imgn[i][j] ^= key[ke]
                    ke+=1

            for x in range(img.shape[0]):
                for y in range(img.shape[1]):
                    finalencimage[x][y][ii] = imgn[x][y]

        finalencimage = Image.fromarray(finalencimage)
        finalencimage.save(f'{savepath}{filename}')
        os.remove(f'{path}{filename}')

In [8]:
from jetbot import bgr8_to_jpeg
image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)
# display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

Error loading module `ublox_gps`: No module named 'serial'


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…